In [ ]:
####応答変数の設定検討####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import matplotlib
import numpy.matlib
import scipy.linalg
import itertools
from scipy.stats import norm  
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import sys

#np.random.seed(98537)

In [ ]:
##データの読み込み
panel_data = pd.read_csv("D:/Statistics/data/stock_price/stock_panel_data.csv", index_col=0)
company_info = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv")
date_range = [np.min(panel_data["日付"]), np.max(panel_data["日付"])]

In [ ]:
#データを絞る
new_panel = panel_data.iloc[np.array([panel_data["日付"] > "2010/01/01"]).reshape(-1)]
new_panel.index = np.arange(new_panel.shape[0])
N = new_panel.shape[0]
N

In [ ]:
#データの設定
price = np.array(new_panel["終値"])
code = np.array(new_panel["コード"], dtype="int")
reference_id = np.array(new_panel["reference_id"], dtype="int")
n = np.unique(reference_id).shape[0]

In [ ]:
#インデックスを作成
index_list = [i for i in range(n)]
no_list = [i for i in range(n)]
max_index = np.repeat(0, n)
for i in range(n):
    if i%100==0:
        print(i)
    index_list[i] = np.array(np.where(reference_id==i)[0], dtype="int")
    no_list[i] = np.arange(index_list[i].shape[0])
    max_index[i] = np.max(index_list[i])
    
#リストを変換
no = np.array(list(itertools.chain(*[np.array(no_list[i]) for i in range(n)])))

In [ ]:
##株価の終値の変動率を算出
#差分を算出するためのインデックス
index_t1 = np.where(no==0)[0]
index_previous = np.array(np.delete(np.arange(N), max_index), dtype="int")
index_behind = np.array(np.delete(np.arange(N), index_t1), dtype="int")

In [ ]:
#変化率を取得
diff_price = np.repeat(0.0, N)
diff_price[index_behind] = price[index_behind] / price[index_previous]

In [ ]:
#データフレームに変換
df = pd.concat([new_panel[["reference_id", "コード", "銘柄名", "業種", "日付", "終値", "出来高"]],
                pd.DataFrame({"diff": diff_price})], axis=1)
out = "D:/Statistics/data/stock_price/stock_price_diff.csv"
df.to_csv(out, sep=",")

In [ ]:
#分布を可視化
mu = np.mean(np.abs(diff_price[index_behind] - 1.0))
sd = np.std(np.abs(diff_price[index_behind] - 1.0)) 
fig = plt.figure(figsize=(15.0, 15.0), dpi=50)
plt.subplots_adjust(wspace=0.2, hspace=0.3)   #余白を設定
ax = fig.add_subplot(2, 2, 1)
ax.hist(diff_price[index_behind] - 1.0, range=(-0.25, 0.25), bins=50)   #変化率のヒストグラム
plt.title("株価の変化率の分布", fontsize=17.5)
ax = fig.add_subplot(2, 2, 2)
ax.hist(np.random.normal(mu, sd, N), range=(-0.25, 0.25), bins=50)
plt.title("理論値での正規分布", fontsize=17.5)

In [ ]:
#外れ値を探す
print(index_behind.shape[0])
print(np.sum(np.abs(diff_price[index_behind]) > 2.0))
print(np.sum(np.abs(diff_price[index_behind]) > 1.75))
print(np.sum(np.abs(diff_price[index_behind]) > 1.5))
print(np.sum(np.abs(diff_price[index_behind]) > 1.035))
print(np.mean(np.abs(diff_price[index_behind] - 1.0)))
print(np.std(np.abs(diff_price[index_behind] - 1.0)))

In [ ]:
#グループごとの集計値の可視化
new_df = df.iloc[index_behind]
new_df["diff"] = np.array(new_df["diff"]) - 1.0
N = new_df.shape[0]
new_df.index = np.arange(N)
grouped = new_df.groupby("銘柄名")

#企業毎の集計値をヒストグラムにする
fig = plt.figure(figsize=(15.0, 15.0), dpi=50)
plt.subplots_adjust(wspace=0.2, hspace=0.3)   #余白を設定
ax = fig.add_subplot(2, 2, 1)
ax.hist(np.array(grouped[["diff"]].mean()).reshape(-1), range=(-0.2, 0.2), bins=50)
plt.title("企業毎の株価変化率の平均分布", fontsize=17.5)
ax = fig.add_subplot(2, 2, 2)
ax.hist(np.array(grouped[["diff"]].std()).reshape(-1), range=(0, 0.15), bins=50)
plt.title("企業毎の株価変化率の標準偏差分布", fontsize=17.5)

In [ ]:
#データの設定
code = np.array(new_df["コード"], dtype="int")
reference_id = np.array(new_df["reference_id"], dtype="int")
n1 = np.unique(reference_id).shape[0]

In [ ]:
#企業のインデックスを作成
index_list1 = [i for i in range(n1)]
no_list1 = [i for i in range(n1)]
max_index1 = np.repeat(0, n1)
for i in range(n1):
    index_list1[i] = np.array(np.where(reference_id==i)[0], dtype="int")
    no_list1[i] = np.arange(index_list1[i].shape[0])
    max_index1[i] = np.max(index_list1[i])
    
#リストを変換
no1 = np.array(list(itertools.chain(*[np.array(no_list1[i]) for i in range(n1)])))

In [ ]:
##データの加工
#時系列成分を加工
unique_date = pd.unique(new_df["日付"])
n2 = unique_date.shape[0]
day_df = pd.DataFrame({"day_id": np.arange(n2), "日付": unique_date})
day_id = np.array(pd.merge(new_df, day_df, on="日付", how="left")["day_id"])

In [ ]:
#日付のインデックスを作成
index_list2 = [i for i in range(n2)]
no_list2 = [i for i in range(n2)]
max_index2 = np.repeat(0, n2)
for i in range(n2):
    index_list2[i] = np.array(np.where(day_id==i)[0], dtype="int")
    no_list2[i] = np.arange(index_list2[i].shape[0])
    max_index2[i] = np.max(index_list2[i])
    
#リストを変換
no2 = np.array(list(itertools.chain(*[np.array(no_list2[i]) for i in range(n2)])))

In [ ]:
#自己回帰成分を抽出
y = np.array(new_df["diff"])
x = np.zeros((N, 2))
index_a = np.delete(np.arange(N), max_index1)
x[:, 0] = np.repeat(1.0, N)
x[index_a+1, 1] = y[index_a]
x_col = x.shape[1]

In [ ]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##アルゴリズムの設定
k = 10
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = 500/keep
disp = 10

In [ ]:
##データの定数
x1_list = [i for i in range(n1)]
x2_list = [i for i in range(n2)]
xx1_list = [i for i in range(n1)]
xx2_list = [i for i in range(n2)]
for i in range(n1):
    x1_list[i] = x[index_list1[i], ]
    xx1_list[i] = np.dot(x1_list[i].T, x1_list[i])
for i in range(n2):
    x2_list[i] = x[index_list2[i], ]
    xx2_list[i] = np.dot(x2_list[i].T, x2_list[i])

In [ ]:
##事前分布の設定
#階層モデルの事前分布
delta = np.repeat(0, k)
nu = 1
V = 0.1 * np.diag(np.ones(k))
s0 = 0.1; v0 = 0.1

#素性ベクトルの事前分布
Lambda = np.repeat(0, x_col) 
kappa = np.diag(np.repeat(100, x_col))
inv_kappa = np.linalg.inv(kappa)

In [ ]:
##初期値の設定
#階層モデルの初期値
alpha1 = np.repeat(0, x_col)
alpha2 = np.repeat(0, x_col)
tau1 = np.diag(np.repeat(0.25, x_col))
tau2 = np.diag(np.repeat(0.25, x_col))
inv_tau1 = np.linalg.inv(tau1)
inv_tau2 = np.linalg.inv(tau1)
Cov1 = np.diag(np.repeat(0.25, k))
Cov2 = np.diag(np.repeat(0.25, k))
inv_Cov1 = np.linalg.inv(Cov1)
inv_Cov2 = np.linalg.inv(Cov2)

#モデルパラメータの初期値
beta1 = np.random.multivariate_normal(alpha1, tau1, n1)
beta2 = np.random.multivariate_normal(alpha2, tau2, n2)
theta1 = np.random.multivariate_normal(np.repeat(0, k), Cov1, n1)
theta2 = np.random.multivariate_normal(np.repeat(0, k), Cov2, n2)
Sigma = 0.1

In [ ]:
##対数尤度の基準値
#1パラメータモデルでの対数尤度
LLst = np.sum(scipy.stats.norm.logpdf(y, np.mean(y), np.std(y)))
print(LLst)

#対数尤度の初期値
x_vec = np.repeat(1.0, x_col)
k_vec = np.repeat(1.0, k)
beta_vec1 = np.dot(x * beta1[reference_id, ], x_vec)
beta_vec2 = np.dot(x * beta2[day_id, ], x_vec)
WH = np.dot(theta1[reference_id, ] * theta2[day_id, ], k_vec)
mu = beta_vec1 + beta_vec2 + WH
sd = np.sum(np.power(y - mu, 2)) / N
LL = np.sum(scipy.stats.norm.logpdf(y, mu, sd))
print(LL)

In [ ]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##beta1をサンプリング
    #モデル誤差を設定
    er = y - beta_vec2 - WH

    for i in range(n1):
        #多変量正規分布のパラメータを設定
        index = index_list1[i]
        x1 = x1_list[i]; xx1 = xx1_list[i]
        xy = np.dot(x1.T, er[index])
        xxv = xx1 + inv_tau1
        inv_xxv = np.linalg.inv(xxv)
        beta_par = np.dot(inv_xxv, xy + np.dot(inv_tau1, alpha1))

        #多変量正規分布から回帰ベクトルをサンプリング
        beta1[i, ] = np.random.multivariate_normal(beta_par, np.power(Sigma, 2)*inv_xxv, 1).reshape(-1)
    beta_vec1 = np.dot(x * beta1[reference_id, ], x_vec)

    ##beta2をサンプリング
    #モデル誤差を設定
    er = y - beta_vec1 - WH

    for i in range(n2):
        #多変量正規分布のパラメータを設定
        index = index_list2[i]
        x2 = x2_list[i]; xx2 = xx2_list[i]
        xy = np.dot(x2.T, er[index])
        xxv = xx2 + inv_tau2
        inv_xxv = np.linalg.inv(xxv)
        beta_par = np.dot(inv_xxv, xy + np.dot(inv_tau2, alpha2))

        #多変量正規分布から回帰ベクトルをサンプリング
        beta2[i, ] = np.random.multivariate_normal(beta_par, np.power(Sigma, 2)*inv_xxv, 1).reshape(-1)
    beta_vec2 = np.dot(x * beta2[day_id, ], x_vec)

    ##theta1をサンプリング
    #モデル誤差を設定
    er = y - beta_vec1 - beta_vec2

    for i in range(n1):
        #多変量正規分布のパラメータを設定
        index = index_list1[i]
        x1 = theta2[day_id[index], ]
        xy = np.dot(x1.T, er[index])
        xxv = np.dot(x1.T, x1) + inv_Cov1
        inv_xxv = np.linalg.inv(xxv)
        theta_par = np.dot(inv_xxv, xy + np.dot(inv_Cov1, np.repeat(0, k)))

        #多変量正規分布から回帰ベクトルをサンプリング
        theta1[i, ] = np.random.multivariate_normal(theta_par, np.power(Sigma, 2)*inv_xxv, 1).reshape(-1)

    ##theta2をサンプリング
    for i in range(n2):
        #多変量正規分布のパラメータを設定
        index =index_list2[i]
        x2 = theta1[reference_id[index], ]
        xy = np.dot(x2.T, er[index])
        xxv = np.dot(x2.T, x2) + inv_Cov2
        inv_xxv = np.linalg.inv(xxv)
        theta_par = np.dot(inv_xxv, xy + np.dot(inv_Cov2, np.repeat(0, k)))

        #多変量正規分布から回帰ベクトルをサンプリング
        theta2[i, ] = np.random.multivariate_normal(theta_par, np.power(Sigma, 2)*inv_xxv, 1).reshape(-1)
    WH = np.dot(theta1[reference_id, ] * theta2[day_id, ], k_vec)


    ##モデルの標準偏差を推定
    mu = beta_vec1 + beta_vec2 + WH
    er = y - mu
    Sigma = np.sum(np.power(er, 2)) / N

    ##サンプリング結果の格納と表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = int(rp/keep)

        if rp%disp==0:
            #サンプリング結果を表示
            print(rp)
            print(np.round(np.array([np.sum(np.power(y - mu, 2)), np.sum(np.power(y - np.mean(y), 2))]), 1))
            print(np.round(np.append(alpha1, alpha2), 3))

In [ ]:
##階層モデルのパラメータを推定
#ランダム効果の階層モデルのパラメータ
alpha1 = np.mean(beta1, axis=0); alpha2 = np.mean(beta2, axis=0)
er1 = beta1 - np.full((n1, x_col), alpha1); er2 = beta2 - np.full((n2, x_col), alpha2)
tau1 = np.dot(er1.T, er1) / n1; tau2 = np.dot(er2.T, er2) / n2
inv_tau1 = np.linalg.inv(tau1); inv_tau2 = np.linalg.inv(tau1)

#特徴ベクトルの階層モデルのパラメータ
er1 = theta1 - np.full((n1, k), np.mean(theta1, axis=0))
er2 = theta2 - np.full((n2, k), np.mean(theta2, axis=0))
Cov1 = np.dot(er1.T, er1) / n1; Cov2 = np.dot(er2.T, er2) / n2
inv_Cov1 = np.linalg.inv(Cov1); inv_Cov2 = np.linalg.inv(Cov2)

In [ ]:
y